In [31]:
import os
from keras.layers import Input, Dense, Conv2D, Conv2DTranspose, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import models
from keras import backend as K
from keras.callbacks import TensorBoard, ModelCheckpoint
from tensorboard.plugins.hparams import api as hp
import tensorflow as tf
import tensorflow as tf
import numpy as np
import pickle
from datetime import datetime
from contextlib import redirect_stdout
import uuid

In [32]:
L = 4
FEATURE_MAP_START = 16
EPOCHS = 32
# quick run of single param or full param sweep. Use True for testing.
QRUN = True
# result dir in the tensorboardraw directory
BOARD_SUB_DIR = 'hp_autoencoder'

In [29]:
if not QRUN:
    HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([15, 50]))
    HP_N_LAYERS = hp.HParam('n_layers', hp.Discrete([2, 3]))
    HP_FEATURE_MAP_STEP = hp.HParam('feature_map_step', hp.Discrete([2, 8, 16]))
    HP_STRIDE_SIZE = hp.HParam('stride', hp.Discrete([1, 2]))
else:
    HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([50]))
    HP_N_LAYERS = hp.HParam('n_layers', hp.Discrete([3]))
    HP_FEATURE_MAP_STEP = hp.HParam('feature_map_step', hp.Discrete([16]))
    HP_STRIDE_SIZE = hp.HParam('stride', hp.Discrete([1]))
    BOARD_SUB_DIR = 'qrun'

# Notes:
* Add `Input` argument `channels_first` when using Z3 data and 
  change the input dimensions.
* You will have to manualy add periodic edges for periodic padding before real runn
* **requires** running the transform notebook to get the data into a format for the auto encoder

In [33]:
with open('pickle_data.pkl', 'rb') as f:
    all_data = pickle.load(f)

all_data = all_data.astype('float32') / 5.0
n_records = len(all_data)
x_train = all_data[: n_records - int(n_records/4)]
x_test = all_data[int(n_records/4) :]
x_train = np.reshape(x_train, (len(x_train), L * 2, L * 2, 1))
x_test = np.reshape(x_test, (len(x_test), L * 2, L * 2, 1))

In [34]:
with tf.summary.create_file_writer(os.path.join('tensorboardraw', BOARD_SUB_DIR)).as_default():
    hp.hparams_config(
        hparams=[HP_BATCH_SIZE, HP_N_LAYERS, HP_FEATURE_MAP_STEP, HP_STRIDE_SIZE],
        metrics=[hp.Metric('binary_crossentropy', display_name='Loss')]
    )

In [35]:
def train_test_model(hparams):
    current_run_id = uuid.uuid4().hex
    
    input_obj = Input(shape=(L*2, L*2, 1))

    x = Conv2D(
        FEATURE_MAP_START,
        (3, 3),
        strides=hparams[HP_STRIDE_SIZE],
        activation='relu',
        padding='same',
        use_bias=True
    )(input_obj)

    for i in range(hparams[HP_N_LAYERS] - 1):
        fm = FEATURE_MAP_START + (i + 1) * hparams[HP_FEATURE_MAP_STEP]
        x = Conv2D(
            fm,
            (3, 3),
            strides=hparams[HP_STRIDE_SIZE],
            activation='relu',
            padding='same',
            use_bias=True
        )(x)
        
    max_fm = fm

    for i in range(hparams[HP_N_LAYERS] - 1):
        fm = max_fm - (i + 1) * hparams[HP_FEATURE_MAP_STEP]
        x = Conv2DTranspose(
            fm,
            (3, 3),
            strides=hparams[HP_STRIDE_SIZE],
            activation='relu',
            padding='same',
            use_bias=True
        )(x)
    
    decoded = Conv2DTranspose(
        1,
        (3, 3),
        strides=hparams[HP_STRIDE_SIZE],
        activation='relu',
        padding='same',
        use_bias=True
    )(x)

    autoencoder = Model(input_obj, decoded)
    #autoencoder.summary()
    autoencoder.compile(
        optimizer='adadelta',
        loss='binary_crossentropy',
        metrics=['binary_crossentropy']
    )

    #log_dir='tensorboardraw/hp_autoencoder/{}_{}'.format(datetime.now(), current_run_id)
    log_dir = os.path.join('tensorboardraw', BOARD_SUB_DIR)
    autoencoder.fit(
        x_train, x_train,
        epochs=EPOCHS,
        batch_size=hparams[HP_BATCH_SIZE],
        shuffle=True,
        validation_data=(x_test, x_test),
        callbacks=[
            TensorBoard(log_dir=log_dir),
            hp.KerasCallback(log_dir, hparams)
        ]
    )
    _, binary_crossentropy = autoencoder.evaluate(x_test, x_test)
    return binary_crossentropy

In [36]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)
        loss = train_test_model(hparams)
        tf.summary.scalar('binary_crossentropy', loss, step=1)

In [25]:
c = 0
for batch_size in HP_BATCH_SIZE.domain.values:
    for n_layers in HP_N_LAYERS.domain.values:
        for f_map_step in HP_FEATURE_MAP_STEP.domain.values:
            for stride in HP_STRIDE_SIZE.domain.values:
                hparams = {
                    HP_BATCH_SIZE: batch_size,
                    HP_N_LAYERS: n_layers,
                    HP_FEATURE_MAP_STEP: f_map_step,
                    HP_STRIDE_SIZE: stride
                }
                c += 1
                run_name = "run-%d" % c
                print('--- Starting trial: %s' % run_name)
                print({h.name: hparams[h] for h in hparams})
                run('tensorboardraw/' + run_name, hparams)

--- Starting trial: run-1
{'batch_size': 15, 'n_layers': 2, 'feature_map_step': 2, 'stride': 1}
Epoch 1/32
500/500 [==============================] - 1s 3ms/step - loss: 3.9853 - binary_crossentropy: 3.9853 - val_loss: 3.9696 - val_binary_crossentropy: 3.9696
Epoch 2/32
500/500 [==============================] - 1s 3ms/step - loss: 3.9504 - binary_crossentropy: 3.9504 - val_loss: 3.9271 - val_binary_crossentropy: 3.9271
Epoch 3/32
500/500 [==============================] - 2s 4ms/step - loss: 3.9089 - binary_crossentropy: 3.9089 - val_loss: 3.8881 - val_binary_crossentropy: 3.8881
Epoch 4/32
500/500 [==============================] - 2s 4ms/step - loss: 3.8729 - binary_crossentropy: 3.8729 - val_loss: 3.8515 - val_binary_crossentropy: 3.8515
Epoch 5/32
500/500 [==============================] - 2s 3ms/step - loss: 3.8374 - binary_crossentropy: 3.8374 - val_loss: 3.8100 - val_binary_crossentropy: 3.8100
Epoch 6/32
500/500 [==============================] - 1s 3ms/step - loss: 3.7773 - b

In [11]:
x_train.shape

(7500, 8, 8, 1)